In [2]:
import scipy.io
import numpy as np
from pathlib import Path
import mne
import torch

In [ ]:
cur_dir = Path.cwd()
path = str(cur_dir.parent) + '/data/DREAMER/'
mat = scipy.io.loadmat(path + 'DREAMER.mat')
print(mat.keys())
print(mat['__header__'])
print(mat['__version__'])
print(mat['__globals__'])

In [ ]:
data, eeg_sr, ecg_sr, eeg_electrodes, n_subjects, n_videos, _, _, _, _  = mat['DREAMER'][0, 0]
eeg_sr = eeg_sr[0, 0]
ecg_sr = ecg_sr[0, 0]
eeg_electrodes = eeg_electrodes[0]
eeg_electrodes = [eeg_electrodes[i][0] for i in range(eeg_electrodes.size)]
n_subjects = n_subjects[0, 0]
n_videos = n_videos[0, 0]
print(data.shape)
print("EEG sampling rate", eeg_sr)
print("ECG sampling rate", ecg_sr)
print("EEG electrodes", eeg_electrodes)
print("Number of subjects", n_subjects)
print("Number of videos", n_videos)

In [6]:
highcut = None
lowcut = .5
order = 3
t = 1.
samples = int(t * eeg_sr)

In [ ]:
for i in range(n_subjects):
    _, _, eeg, _, val, aro, dom = data[0, i][0][0]
    emotions = {'valence': val, 'arousal': aro, 'dominance': dom}
    baseline_eeg, stimuli_eeg = eeg[0, 0]
    for j in range(n_videos):
        file_name = f"S{i+1:02}R{j+1:02}.edf.raw.fif"
        annotation_file_name = f"S{i+1:02}R{j+1:02}.edf.events"

        stimuli_eeg_j = stimuli_eeg[j, 0]
        baseline_eeg_j = baseline_eeg[j, 0]
        stimuli_eeg_j -= np.mean(baseline_eeg_j, axis=0)
        stimuli_eeg_j /= np.std(baseline_eeg_j, axis=0)
        info = mne.create_info(ch_names=eeg_electrodes, sfreq=eeg_sr, ch_types=['eeg']*len(eeg_electrodes))

        raw_stim = mne.io.RawArray(stimuli_eeg_j.T, info)
        raw_stim.filter(lowcut, highcut, 
                                method='iir', 
                                iir_params=dict(order=order, ftype='butterworth'))
        l = stimuli_eeg_j.shape[0]

        for emotion in emotions:

            annotation_desc = [emotion[0].upper() + f'{emotions[emotion][j, 0]}' for _ in range(l//samples)]
            annotation_onset = [k*t for k in range(l//samples)]
            annotation_duration = [samples/eeg_sr for _ in range(l//samples)]
            raw_stim.set_annotations(mne.Annotations(onset=annotation_onset, duration=annotation_duration,
                                                    description=annotation_desc))
            raw_stim.save(path + 'files_' + emotion + f'/S{i+1:02}/' + file_name, overwrite=True)
            raw_stim.annotations.save(path + f'files_' + emotion + f'/S{i+1:02}/' + annotation_file_name, overwrite=True)

In [ ]:
for i in range(n_subjects):
    _, _, eeg, _, val, aro, dom = data[0, i][0][0]
    baseline_eeg, stimuli_eeg = eeg[0, 0]
    for j in range(n_videos):
        file_name = f"S{i+1:02}R{j+1:02}.edf.raw.fif"
        annotation_file_name = f"S{i+1:02}R{j+1:02}.edf.events"

        stimuli_eeg_j = stimuli_eeg[j, 0]
        baseline_eeg_j = baseline_eeg[j, 0]
        stimuli_eeg_j -= np.mean(baseline_eeg_j, axis=0)
        stimuli_eeg_j /= np.std(baseline_eeg_j, axis=0)
        info = mne.create_info(ch_names=eeg_electrodes, sfreq=eeg_sr, ch_types=['eeg']*len(eeg_electrodes))

        raw_stim = mne.io.RawArray(stimuli_eeg_j.T, info)
        raw_stim.filter(lowcut, highcut, 
                                method='iir', 
                                iir_params=dict(order=order, ftype='butterworth'))
        l = stimuli_eeg_j.shape[0]

        annotation_desc = [f'A{aro[j, 0]}' for _ in range(l//samples)]
        annotation_onset = [k*2. for k in range(l//samples)]
        annotation_duration = [samples/eeg_sr for _ in range(l//samples)]
        raw_stim.set_annotations(mne.Annotations(onset=annotation_onset, duration=annotation_duration,
                                                description=annotation_desc))
        raw_stim.save(path + f'files_arousal/S{i+1:02}/' + file_name, overwrite=True)
        raw_stim.annotations.save(path + f'files_arousal/S{i+1:02}/' + annotation_file_name, overwrite=True)

In [ ]:
X = []
# t_raw = None
# t_raw_2 = None

for i in range(n_subjects):
    _, _, eeg, _, val, aro, dom = data[0, i][0][0]
    baseline_eeg, stimuli_eeg = eeg[0, 0]
    for j in range(n_videos):
        file_name = f"S{i+1:02}R{j+1:02}.edf.raw.fif"
        annotation_file_name = f"S{i+1:02}R{j+1:02}.edf.events"

        stimuli_eeg_j = stimuli_eeg[j, 0]
        baseline_eeg_j = baseline_eeg[j, 0]
        stimuli_eeg_j -= np.mean(baseline_eeg_j, axis=0)
        stimuli_eeg_j /= np.std(baseline_eeg_j, axis=0)
        info = mne.create_info(ch_names=eeg_electrodes, sfreq=eeg_sr, ch_types=['eeg']*len(eeg_electrodes))

        raw_stim = mne.io.RawArray(stimuli_eeg_j.T, info)
        raw_stim.filter(lowcut, highcut, 
                                method='iir', 
                                iir_params=dict(order=order, ftype='butterworth'))
        l = stimuli_eeg_j.shape[0]

        # t_raw = raw_stim

        annotation_desc = [f'D{dom[j, 0]}' for _ in range(l//samples)]
        annotation_onset = [k*2. for k in range(l//samples)]
        annotation_duration = [samples/eeg_sr for _ in range(l//samples)]
        raw_stim.set_annotations(mne.Annotations(onset=annotation_onset, duration=annotation_duration,
                                                description=annotation_desc))
        raw_stim.save(path + f'files_dominance/S{i+1:02}/' + file_name, overwrite=True)
        raw_stim.annotations.save(path + f'files_dominance/S{i+1:02}/' + annotation_file_name, overwrite=True)
        # t_raw_2 = raw_stim

        for k in range(l//samples):
            X.append(torch.tensor(stimuli_eeg_j[l-((k+1)*samples):l-(k*samples), :].T, dtype=torch.float32))

X = torch.stack(X)

In [ ]:
print("Max: ", torch.max(X))
print("Min: ", torch.min(X))

In [ ]:
cur_dir = Path.cwd()
test_path = str(cur_dir.parent) + '/data/MMIDB/S001/'
test = mne.io.read_raw_edf(test_path + 'S001R03.edf')

In [ ]:
print(vars(test.annotations))
print(test.annotations)
print(test.annotations.description)
print(test.annotations.onset)
print(test.annotations.duration)
print(test.annotations.onset*test.info['sfreq'])
print(test.annotations.onset*256)
print(test.info['sfreq'])

In [ ]:
test

In [ ]:
t_raw.annotations

In [ ]:
t_raw_2.annotations